<a href="https://colab.research.google.com/github/QuantumZeroAR/Capacitacion_AI/blob/main/C4.0_Regresion_Lineal_Precio_Casas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Regresion lineal costo casas en California.**

www.kaggle.com
Usaremos dataset: https://www.kaggle.com/datasets/camnugent/california-housing-prices

Precio de las casas en el estado de California


1. longitude: medida de qué tan al oeste está una casa; un valor más alto es más al oeste

2. latitude: medida de qué tan al norte está una casa; un valor más alto es más al norte

3. housingMedianAge: Medad media de una casa dentro de una cuadra; un número más bajo es un edificio más nuevo

4. totalRooms: número total de habitaciones dentro de una cuadra

5. totalBedrooms: número total de habitaciones para dormir dentro de una cuadra

6. population: número total de personas que residen dentro de una cuadra

7. households: Total de casas en la cuadra.

8. medianIncome: ingreso medio para hogares dentro de una cuadra de casas (medido en decenas de miles de dólares estadounidenses, es decir: x10000)

9. medianHouseValue: valor medio de la casa para hogares dentro de una cuadra (medido en dólares estadounidenses)

10. oceanProximity: ubicación de la casa con respecto al océano. 1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND

Son bloques de casas por latitud y longitud. Por eso se expresan los valores en promedios de cada bloque.




In [1]:
import pandas as pd

datos = pd.read_csv("housing.csv")

In [ ]:
datos.head()

In [ ]:
#Estos datos mas adelante tendremos que pasarlos a numericos
datos["ocean_proximity"].value_counts()

In [ ]:
#Total_bedrooms mas adelante hay que ver que hacer con los valores faltantes
datos.info()

In [ ]:
#Los ingresos promedios van de U$S5.000 a U$S150.000
datos.describe()

In [ ]:
#Vamos a graficar enforma automatica algunos histogramas.
# ver que antiguedad posiblemente esta cargando todo lo que es mayor a 52 en 52.
# Lo mismo pasa con el valor de las casas he ingresos. Hay que ver como manejar esto

datos.hist(figsize=(15,8), bins=30, edgecolor="black")

In [ ]:
import seaborn as sb
sb.scatterplot(x="latitude", y="longitude", data=datos)

In [ ]:
sb.scatterplot(x="latitude", y="longitude", data=datos, hue="median_house_value")

In [ ]:
sb.scatterplot(x="latitude", y="longitude", data=datos, hue="median_house_value", palette="coolwarm")

#sizes=datos["population"]/100

In [ ]:
#vamos a filtrar los datos en el grafico para mostrar solo los que mas altos ingresos tienen.
sb.scatterplot(x="latitude", y="longitude", data=datos[(datos.median_income > 14)], hue="median_house_value", palette="coolwarm")

In [ ]:
datos.info()

In [17]:
#Eliminamos na directamente ya que son pocos datos.

datos_na = datos.dropna()

In [ ]:
datos_na.info()

In [ ]:
#Convertir la caracteística categórica a numérica
#Proximidad al oceano
datos_na["ocean_proximity"]


In [ ]:
datos_na["ocean_proximity"].value_counts()
# 1,2,3,4,5

In [21]:
#Dummies / One-Hot Encoding
# NEAR BAY   INLAND   NEAR OCEAN
#    1          0         0
#    0          0         1

dummies = pd.get_dummies(datos_na["ocean_proximity"], dtype=int)

In [22]:
datos_na = datos_na.join(dummies)

In [ ]:
datos_na.head()

In [24]:
datos_na = datos_na.drop(["ocean_proximity"], axis=1)

In [ ]:
datos_na.head()

In [ ]:
#Analisis, nuevas caracteristicas

In [ ]:

datos_na.corr()

In [ ]:
#Mostramos correlacion de valores y relaciones.

sb.set(rc={'figure.figsize': (15,8)})   #Especifico el tamaño de los cuadros. Se puede cambiar de acuerdo al monitor
sb.heatmap(datos_na.corr(), annot=True, cmap="YlGnBu")

In [ ]:
#Listamos correlacion referida al valor medio de las casas.
#Con esto podemos determinar que hay una relacion fuerte entre el costo de la casa
#y el ingreso promedio.

datos_na.corr()[["median_house_value"]].sort_values(by="median_house_value", ascending=False)

In [ ]:
# Graficamos la relacion y podemos ver que hay una relacion bastante lineal.
# podriamos utilizar un modelo de regresion lineal.

sb.scatterplot(x=datos_na["median_house_value"], y=datos_na["median_income"])

In [34]:
#Creamos una nueva caracteristica que sea habitaciones para dormir dividido total de habitaciones
datos_na["bedroom_ratio"] = datos_na["total_bedrooms"] / datos_na["total_rooms"]

In [ ]:
#Vemos que la nueva caracteristica tiene una relacion mas fuerte que las 2 por su parte.
#Es decir que la casa vale mas si tiene mas habitaciones que no sean para dormir

sb.set(rc={'figure.figsize': (15,8)})
sb.heatmap(datos_na.corr(), annot=True, cmap="YlGnBu")

In [36]:
#Separar las caracteristicas de la etiqueta
X = datos_na.drop(["median_house_value"], axis=1)
y = datos_na["median_house_value"]

In [37]:
#Separar los datos en 2 partes: Conjunto de entrenamiento y uno de pruebas
from sklearn.model_selection import train_test_split

X_ent, X_pru, y_ent, y_pru = train_test_split(X, y, test_size=.2)

In [38]:
from sklearn.linear_model import LinearRegression

modelo=LinearRegression()

In [ ]:
modelo.fit(X_ent, y_ent)

In [40]:
predicciones = modelo.predict(X_pru)

In [ ]:
comparativa = {"Prediccion": predicciones, "Valor Real": y_pru}
pd.DataFrame(comparativa)

In [ ]:
from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='Prediccion', y='Valor Real', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
#Overfitting sobreajuste
print(modelo.score(X_ent, y_ent))
print(modelo.score(X_pru, y_pru))

In [46]:
#Error
#Tomo los datos, saca la diferencia y la eleva al cuadrado
from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y_pru, predicciones)

In [ ]:
mse

In [48]:
#Obtengo la raiz cuadrada de MSE para visualizar el error en escala real

rmse = np.sqrt(mse)

In [ ]:
rmse